Segmentez des clients d'un site de e-commerce
==========================================

![logo](https://olist.com/wp-content/uploads/2018/04/Logo-01.png)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import seaborn as sns
from sklearn.cluster import DBSCAN, KMeans, AgglomerativeClustering
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm

from src.visualization.visualize import group_analysis
sns.set(font_scale=1.)

In [ ]:
data = pd.read_csv('../data/processed/dataset.csv')

**Note**

Les résultats ci-dessous peuvent varier d'une exécution à l'autre.

## Transformation log

In [ ]:
col_to_log = ['monetary', 'clothing', 'high-tech', 'home', 'other']
for col in col_to_log:
    data[col] = data[col].apply(np.log1p)

## Split manuel entre client régulier et client ponctuel

In [ ]:
ponctual = data[data['frequency'] == 1].copy()
loyal = data[data['frequency'] > 1].copy()

In [ ]:
ponctual

In [ ]:
ponctual.shape[0] / data.shape[0]

DBSCAN
--------

### Données complètes

In [ ]:
features = ['recency',
            'monetary',
            'item_per_c',
            'clothing',
            'high-tech',
            'hobbies',
            'home',
            'other',
            'review_score']

In [ ]:
X = data.loc[:, features + ['frequency']].values
X = StandardScaler().fit_transform(X)

In [ ]:
db = DBSCAN(eps=1, min_samples=100).fit(X)
labels = db.labels_
core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
core_samples_mask[db.core_sample_indices_] = True

In [ ]:
n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
n_noise_ = list(labels).count(-1)
print('Estimated number of clusters: %d' % n_clusters_)
print('Estimated number of noise points: %d' % n_noise_)

In [ ]:
data.loc[:, 'group'] = labels

In [ ]:
group_analysis(data)

### Clients pontuels

In [ ]:
X = ponctual.loc[:, features].values
X = StandardScaler().fit_transform(X)

In [ ]:
db = DBSCAN(eps=1.0, min_samples=1000).fit(X)
labels = db.labels_
core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
core_samples_mask[db.core_sample_indices_] = True

In [ ]:
n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
n_noise_ = list(labels).count(-1)
print('Estimated number of clusters: %d' % n_clusters_)
print('Estimated number of noise points: %d' % n_noise_)

In [ ]:
ponctual.loc[:, 'group'] = labels

In [ ]:
group_analysis(ponctual)

In [ ]:
%%time
print("Silhouette Coefficient: %0.3f"
      % metrics.silhouette_score(X, labels))

### Clients fidèles

In [ ]:
X = loyal.loc[:, features + ['frequency']].values
X = StandardScaler().fit_transform(X)

In [ ]:
db = DBSCAN(eps=0.6, min_samples=10).fit(X)
labels = db.labels_
core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
core_samples_mask[db.core_sample_indices_] = True

In [ ]:
n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
n_noise_ = list(labels).count(-1)
print('Estimated number of clusters: %d' % n_clusters_)
print('Estimated number of noise points: %d' % n_noise_)

In [ ]:
loyal.loc[:, 'group'] = labels

In [ ]:
group_analysis(loyal)

In [ ]:
%%time
print("Silhouette Coefficient: %0.3f"
      % metrics.silhouette_score(X, labels))

## Kmeans

### Données complètes

In [ ]:
X = data.loc[:, features].values
X = StandardScaler().fit_transform(X)

In [ ]:
scores = list()
for n in tqdm(np.arange(2, 11)):
    kmeans = KMeans(n_clusters=n).fit(X)
    labels = kmeans.labels_
    scores.append(metrics.silhouette_score(X, labels))

plt.plot(np.arange(2, 11), scores)

In [ ]:
kmeans = KMeans(n_clusters=7).fit(X)
labels = kmeans.labels_
data.loc[:, 'group'] = labels
group_analysis(data)

In [ ]:
kmeans = KMeans(n_clusters=9).fit(X)
labels = kmeans.labels_
data.loc[:, 'group'] = labels
group_analysis(data)

In [ ]:
%%time
print("Silhouette Coefficient: %0.3f"
      % metrics.silhouette_score(X, labels))

### Segmentation des clients pontuels

In [ ]:
X = ponctual.loc[:, features].values
X = StandardScaler().fit_transform(X)

In [ ]:
scores = list()
for n in tqdm(np.arange(2, 11)):
    kmeans = KMeans(n_clusters=n).fit(X)
    labels = kmeans.labels_
    scores.append(metrics.silhouette_score(X, labels))

plt.plot(np.arange(2, 11), scores)

In [ ]:
kmeans = KMeans(n_clusters=7).fit(X)
labels = kmeans.labels_

In [ ]:
ponctual.loc[:, 'group'] = labels

In [ ]:
group_analysis(ponctual)

In [ ]:
%%time
print("Silhouette Coefficient: %0.3f"
      % metrics.silhouette_score(X, labels))

### segmentation clients réguliers

In [ ]:
X = loyal.loc[:, features + ['frequency']].values
X = StandardScaler().fit_transform(X)

In [ ]:
scores = list()
for n in np.arange(2, 11):
    kmeans = KMeans(n_clusters=n).fit(X)
    labels = kmeans.labels_
    scores.append(metrics.silhouette_score(X, labels))

plt.plot(np.arange(2, 11), scores)

In [ ]:
kmeans = KMeans(n_clusters=8).fit(X)
labels = kmeans.labels_

In [ ]:
loyal.loc[:, 'group'] = labels

In [ ]:
group_analysis(loyal)

In [ ]:
%%time
print("Silhouette Coefficient: %0.3f"
      % metrics.silhouette_score(X, labels))

Agglomerative_clustering
--------------------------------------

### Données complètes

In [ ]:
X = data.loc[:, features + ['frequency']].values
X = StandardScaler().fit_transform(X)

scores = list()
for n in tqdm(np.arange(2, 13)):
    agg = AgglomerativeClustering(n_clusters=n).fit(X)
    labels = agg.labels_
    scores.append(metrics.silhouette_score(X, labels))

plt.plot(np.arange(2, 13), scores)

In [ ]:
agg = AgglomerativeClustering(n_clusters=11).fit(X)
labels = agg.labels_
data.loc[:, 'group'] = labels
group_analysis(data)

In [ ]:
%%time
print("Silhouette Coefficient: %0.3f"
      % metrics.silhouette_score(X, labels))

### Segmentation clients ponctuels

In [ ]:
X = ponctual.loc[:, features].values
X = StandardScaler().fit_transform(X)

scores = list()
for n in tqdm(np.arange(2, 13)):
    agg = AgglomerativeClustering(n_clusters=n).fit(X)
    labels = agg.labels_
    scores.append(metrics.silhouette_score(X, labels))

plt.plot(np.arange(2, 13), scores)

In [ ]:
agg = AgglomerativeClustering(n_clusters=10)
agg.fit(X)

In [ ]:
ponctual.loc[:, 'group'] = agg.labels_

In [ ]:
group_analysis(ponctual)

In [ ]:
%%time
print("Silhouette Coefficient: %0.3f"
      % metrics.silhouette_score(X, labels))

### Clients réguliers

In [ ]:
X = loyal.loc[:, features + ['frequency']].values
X = StandardScaler().fit_transform(X)

scores = list()
for n in tqdm(np.arange(2, 11)):
    agg = AgglomerativeClustering(n_clusters=n).fit(X)
    labels = agg.labels_
    scores.append(metrics.silhouette_score(X, labels))

plt.plot(np.arange(2, 11), scores)

In [ ]:
agg = AgglomerativeClustering(n_clusters=2)
agg.fit(X)

In [ ]:
loyal.loc[:, 'group'] = agg.labels_

In [ ]:
group_analysis(loyal)

In [ ]:
%%time
print("Silhouette Coefficient: %0.3f"
      % metrics.silhouette_score(X, agg.labels_))